In [86]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [77]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, make_scorer, r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

In [15]:
mat_data = pd.read_csv('student-mat.csv')
por_data = pd.read_csv('student-por.csv')


In [16]:
#combined_data
df_data = pd.concat([mat_data, por_data])
df_data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


In [17]:
#identifying binary columns
binary_cols=[]
for i in df_data.columns:
    if df_data[i].nunique()==2:
        binary_cols.append(i)
binary_cols

['school',
 'sex',
 'address',
 'famsize',
 'Pstatus',
 'schoolsup',
 'famsup',
 'paid',
 'activities',
 'nursery',
 'higher',
 'internet',
 'romantic']

In [18]:
# turning categorical columns into binary
df_data = pd.get_dummies(df_data, columns=['Mjob', 'Fjob', 'reason', 'guardian'])
df_data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,traveltime,studytime,...,Fjob_other,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_other,reason_reputation,guardian_father,guardian_mother,guardian_other
0,GP,F,18,U,GT3,A,4,4,2,2,...,False,False,True,True,False,False,False,False,True,False
1,GP,F,17,U,GT3,T,1,1,1,2,...,True,False,False,True,False,False,False,True,False,False
2,GP,F,15,U,LE3,T,1,1,1,2,...,True,False,False,False,False,True,False,False,True,False
3,GP,F,15,U,GT3,T,4,2,1,3,...,False,True,False,False,True,False,False,False,True,False
4,GP,F,16,U,GT3,T,3,3,1,2,...,True,False,False,False,True,False,False,True,False,False


In [75]:
#df_data.info()

In [19]:
df_data.describe()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
count,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000
mean,16.726054,2.603448,2.387931,1.522989,1.970307,0.264368,3.935824,3.201149,3.156130,1.494253,2.284483,3.543103,4.434866,11.213602,11.246169,11.341954
std,1.239975,1.124907,1.099938,0.731727,0.834353,0.656142,0.933401,1.031507,1.152575,0.911714,1.285105,1.424703,6.210017,2.983394,3.285071,3.864796
min,15.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,16.000000,2.000000,1.000000,1.000000,1.000000,0.000000,4.000000,3.000000,2.000000,1.000000,1.000000,3.000000,0.000000,9.000000,9.000000,10.000000
50%,17.000000,3.000000,2.000000,1.000000,2.000000,0.000000,4.000000,3.000000,3.000000,1.000000,2.000000,4.000000,2.000000,11.000000,11.000000,11.000000
75%,18.000000,4.000000,3.000000,2.000000,2.000000,0.000000,5.000000,4.000000,4.000000,2.000000,3.000000,5.000000,6.000000,13.000000,13.000000,14.000000
max,22.000000,4.000000,4.000000,4.000000,4.000000,3.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,75.000000,19.000000,19.000000,20.000000


In [20]:
# Seperating the variables and the output values
X = df_data.drop('G3', axis=1)
Y = df_data['G3']

In [21]:
X

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,traveltime,studytime,...,Fjob_other,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_other,reason_reputation,guardian_father,guardian_mother,guardian_other
0,GP,F,18,U,GT3,A,4,4,2,2,...,False,False,True,True,False,False,False,False,True,False
1,GP,F,17,U,GT3,T,1,1,1,2,...,True,False,False,True,False,False,False,True,False,False
2,GP,F,15,U,LE3,T,1,1,1,2,...,True,False,False,False,False,True,False,False,True,False
3,GP,F,15,U,GT3,T,4,2,1,3,...,False,True,False,False,True,False,False,False,True,False
4,GP,F,16,U,GT3,T,3,3,1,2,...,True,False,False,False,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,MS,F,19,R,GT3,T,2,3,1,3,...,True,False,False,True,False,False,False,False,True,False
645,MS,F,18,U,LE3,T,3,1,1,2,...,False,True,False,True,False,False,False,False,True,False
646,MS,F,18,U,GT3,T,1,1,2,2,...,True,False,False,True,False,False,False,False,True,False
647,MS,M,17,U,LE3,T,3,1,2,1,...,False,True,False,True,False,False,False,False,True,False


In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)


In [55]:
le = LabelEncoder()
for i in binary_cols:
    le.fit(X_train[i])
    X_train[i] = le.transform(X_train[i])
    X_test[i] = le.fit_transform(X_test[i])

In [56]:
X_train

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,traveltime,studytime,...,Fjob_other,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_other,reason_reputation,guardian_father,guardian_mother,guardian_other
248,0,1,18,0,1,1,3,3,1,2,...,False,True,False,True,False,False,False,False,True,False
306,0,1,20,1,0,0,3,2,1,1,...,True,False,False,True,False,False,False,False,False,True
284,0,1,18,1,0,1,2,1,1,1,...,False,True,False,False,False,True,False,False,True,False
371,1,1,18,0,1,1,1,2,3,1,...,False,True,False,False,False,True,False,True,False,False
30,0,1,15,1,0,1,4,4,1,2,...,False,True,False,False,True,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,0,0,15,1,0,1,4,2,1,3,...,True,False,False,False,False,False,True,False,True,False
330,0,1,18,1,1,1,2,2,1,4,...,True,False,False,True,False,False,False,False,True,False
71,0,1,15,1,0,1,4,2,1,4,...,True,False,False,True,False,False,False,False,True,False
121,0,1,15,1,0,1,2,2,1,4,...,False,True,False,False,True,False,False,True,False,False


In [57]:
X_test.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,traveltime,studytime,...,Fjob_other,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_other,reason_reputation,guardian_father,guardian_mother,guardian_other
576,1,1,18,0,0,1,3,2,2,1,...,True,False,False,True,False,False,False,False,True,False
280,0,1,17,1,1,0,4,1,2,1,...,True,False,False,False,True,False,False,False,True,False
141,0,0,16,0,0,1,1,1,4,2,...,True,False,False,True,False,False,False,False,True,False
429,1,1,16,0,1,0,4,4,1,2,...,True,False,False,False,True,False,False,False,True,False
249,0,1,16,1,0,1,3,2,2,3,...,True,False,False,False,False,False,True,False,True,False


In [58]:
#scale the data to equalise the values of each feature between 0 and 1
scaler = MinMaxScaler()
#scaler.fit(X_train)

X_train_scaled = scaler.fit_transform(X_train)
#X_train_scaled = pd.dataframe(X_train_scaled)
#X_train_scaled

In [59]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_train_scaled

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,traveltime,studytime,...,Fjob_other,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_other,reason_reputation,guardian_father,guardian_mother,guardian_other
0,0.0,1.0,0.428571,0.0,1.0,1.0,0.75,0.75,0.000000,0.333333,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.714286,1.0,0.0,0.0,0.75,0.50,0.000000,0.000000,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,1.0,0.428571,1.0,0.0,1.0,0.50,0.25,0.000000,0.000000,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,1.0,1.0,0.428571,0.0,1.0,1.0,0.25,0.50,0.666667,0.000000,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,0.0,1.0,0.000000,1.0,0.0,1.0,1.00,1.00,0.000000,0.333333,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,0.0,0.0,0.000000,1.0,0.0,1.0,1.00,0.50,0.000000,0.666667,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
779,0.0,1.0,0.428571,1.0,1.0,1.0,0.50,0.50,0.000000,1.000000,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
780,0.0,1.0,0.000000,1.0,0.0,1.0,1.00,0.50,0.000000,1.000000,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
781,0.0,1.0,0.000000,1.0,0.0,1.0,0.50,0.50,0.000000,1.000000,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [60]:
#X_train.head()
#X_test.head()

In [61]:
# Train random forest model
rf = RandomForestRegressor(n_estimators=80, random_state=10)
rf.fit(X_train_scaled, y_train)

RandomForestRegressor(n_estimators=80, random_state=10)

In [106]:
# Finding feature importances
importance = pd.DataFrame({'Feature': X_train_scaled.columns, 'Importance': rf.feature_importances_})
top_30_features = importance.sort_values('Importance', ascending=False)[:30]
top_30_features

,Feature,Importance
27,G2,0.827726
25,absences,0.057338
26,G1,0.010737
21,goout,0.008521
9,studytime,0.007951
24,health,0.005998
2,age,0.005589
20,freetime,0.004891
1,sex,0.004540
19,famrel,0.004073


In [83]:
# Compute feature importances
# importances = rf.feature_importances_
# std = np.std([tree.feature_importances_ for tree in rf.estimators_], axis=0)
# indices = np.argsort(importances)[::-1]

In [101]:
#Print feature ranking
# print("Feature ranking:")
# for f in range(X.shape[1]):
#     print(f"{f+1}. {X.columns[indices[f]]} ({importances[indices[f]]})")

In [88]:
# Plot feature importances
# plt.figure()
# plt.title("Feature importances")
# plt.bar(range(X.shape[1]), importances[indices], yerr=std[indices], align="center")
# plt.xticks(range(X.shape[1]), X.columns[indices], rotation='vertical')
# plt.xlim([-1, X.shape[1]])
# plt.tight_layout()
# plt.show()

In [66]:
y_pred = rf.predict(scaler.fit_transform(X_test))
# Evaluate model performance
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 1.698872310035355


C:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [67]:
r2 = cross_val_score(rf, X_train_scaled, y_train, cv=5, scoring=make_scorer(r2_score)).mean()
r2

0.8577491243177382

In [79]:
#y_pred
#y_test

In [74]:
np_test=np.array(y_test)
np_pred=np.array(y_pred)
compare_df=pd.DataFrame(data=[np_test,np_pred]).T 
compare_df.columns=['test_scores','predicted_scores']
compare_df.head(10)

,test_scores,predicted_scores
0,11.0,9.7250
1,8.0,6.1000
2,13.0,13.2875
3,11.0,11.0375
4,12.0,13.0875
5,11.0,11.7750
6,0.0,2.8250
7,11.0,10.9125
8,15.0,14.3500
9,13.0,13.4375
